In [1]:
import codecs
import jieba as jb
import re
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import random
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
pip install jieba

Note: you may need to restart the kernel to use updated packages.


In [3]:
#中文分局
def SentsTokenizer4Ch(text):
    #sentences = re.split('(。|！|\!|\.|？|\?)',text)
    sentences = re.split('(。|！|\!|？|\?)',text)
    new_sents = []
    for i in range(int(len(sentences)/2)):
        sent = sentences[2*i] + sentences[2*i+1]
        sent = sent.replace('\r\n','')
        sent = sent.strip()
        new_sents.append(sent)
    return new_sents

#删除所有符号,只保留字母、数字和中文
def remove_punctuation(line):
    #line = str(line)
    if line.strip()=='':
        return ''
    rule = re.compile(u"[^a-zA-Z0-9\u4E00-\u9FA5]")
    line = rule.sub('',line)
    return line
 
#中文停用词
def stopwordslist(filepath):  
    stopwords = [line.strip() for line in open(filepath, 'r', encoding='utf8',errors='ignore').readlines()]  
    return stopwords  


#加载停用词
stopwords = stopwordslist("/Users/lilyhe/Documents/NLP_BERT/Chatbot_chinese/data/chineseStopWords.txt")

In [4]:
myfile = "/Users/lilyhe/Documents/NLP_BERT/Chatbot_chinese/data/Covid19.txt"

text = codecs.open(myfile,encoding="utf16").read()
#with open(myfile, 'rb') as f: text = f.read()

sent_tokens = SentsTokenizer4Ch(text)
sent_tokens[:10]

['7月3日0—24时，31个省（自治区、直辖市）和新疆生产建设兵团报告新增确诊病例72例。',
 '其中境外输入病例31例（广东9例，上海7例，福建5例，内蒙古2例，浙江2例，北京1例，天津1例，辽宁1例，江西1例，河南1例，四川1例），含7例由无症状感染者转为确诊病例（广东4例，天津1例，福建1例，河南1例）；本土病例41例（安徽29例，江苏4例，山东4例，上海2例，福建1例，广东1例），含2例由无症状感染者转为确诊病例（安徽1例，山东1例）。',
 '无新增死亡病例。',
 '无新增疑似病例。',
 '当日新增治愈出院病例50例，其中境外输入病例32例，本土病例18例（内蒙古7例，上海6例，北京5例），解除医学观察的密切接触者3759人，重症病例与前一日持平。',
 '境外输入现有确诊病例299例（无重症病例），无现有疑似病例。',
 '累计确诊病例19485例，累计治愈出院病例19186例，无死亡病例。',
 '截至7月3日24时，据31个省（自治区、直辖市）和新疆生产建设兵团报告，现有确诊病例532例（无重症病例），累计治愈出院病例220165例，累计死亡病例5226例，累计报告确诊病例225923例，无现有疑似病例。',
 '累计追踪到密切接触者4282315人，尚在医学观察的密切接触者61500人。']

In [5]:
#删除除字母,数字，汉字以外的所有符号
df = pd.DataFrame(sent_tokens, columns=['sent'])
df['clean_set']=  df['sent'].apply(remove_punctuation)

#分词，并过滤停用词
# df['cut_sent'] = df['clean_set'].apply(lambda x: " ".join([w for w in list(jb.cut(x)) if w not in stopwords]))
df['cut_sent'] = df['clean_set'].apply(lambda x: " ".join([w for w in list(jb.cut(x))]))
df.head()

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/lb/f39r117d24l1d72_dd9psjcc0000gn/T/jieba.cache
Loading model cost 0.637 seconds.
Prefix dict has been built successfully.


,sent,clean_set,cut_sent
0,7月3日0—24时，31个省（自治区、直辖市）和新疆生产建设兵团报告新增确诊病例72例。,7月3日024时31个省自治区直辖市和新疆生产建设兵团报告新增确诊病例72例,7 月 3 日 024 时 31 个省 自治区 直辖市 和 新疆生产建设兵团 报告 新增 确...
1,其中境外输入病例31例（广东9例，上海7例，福建5例，内蒙古2例，浙江2例，北京1例，天津1...,其中境外输入病例31例广东9例上海7例福建5例内蒙古2例浙江2例北京1例天津1例辽宁1例江西...,其中 境外 输入 病例 31 例 广东 9 例 上海 7 例 福建 5 例 内蒙古 2 例 ...
2,无新增死亡病例。,无新增死亡病例,无 新增 死亡 病例
3,无新增疑似病例。,无新增疑似病例,无 新增 疑似病例
4,当日新增治愈出院病例50例，其中境外输入病例32例，本土病例18例（内蒙古7例，上海6例，北...,当日新增治愈出院病例50例其中境外输入病例32例本土病例18例内蒙古7例上海6例北京5例解除...,当日 新增 治愈 出院 病例 50 例 其中 境外 输入 病例 32 例 本土 病例 18 ...


In [6]:
GREETING_INPUTS = ("你好", "hi", "有人", "在吗", "嗨","在不在","有人吗",'在','有人')
GREETING_RESPONSES = ["你好",  "我在", "请问,有什么可以帮您的吗?", "你好，我在", "你好，很高兴见到你！"]

def greeting(sentence):
    rule = re.compile(u"[^a-zA-Z0-9\u4E00-\u9FA5]")
    text = rule.sub('',sentence)
    if text in GREETING_INPUTS:
        return random.choice(GREETING_RESPONSES)
    
    wordlist =  [w for w in jb.cut(sentence)]
    for word in wordlist:
        if word in GREETING_INPUTS:
            return random.choice(GREETING_RESPONSES)

In [7]:
greeting('有人在吗？')

'你好，我在'

In [8]:
def response(user_response):
    robo_response=''
    user_response  = remove_punctuation(user_response)
    user_response= " ".join([w for w in list(jb.cut(user_response)) if w not in stopwords])
    cut_sent = df.cut_sent.values.tolist()
    cut_sent.append(user_response)
    tfidf = TfidfVectorizer()

    tfidf_vec = tfidf.fit_transform(cut_sent)

    cos_sim = cosine_similarity(tfidf_vec[-1], tfidf_vec)
    idx=cos_sim.argsort()[0][-2]
    flat = cos_sim.flatten()
    flat.sort()
    req_tfidf = flat[-2]

    if(req_tfidf==0):
        robo_response=robo_response+"对不起,我不理解您的意思,我还是个小白...!"
        return(robo_response)
    else:
        robo_response = robo_response+df.sent.values[idx]
        return(robo_response) 

In [ ]:
flag=True
print("机器人: 我的名字叫小白. 我可以回答您关于Covid最新政策的问题. 如果您想退出, 请输入:bye !")

while(flag==True):
    user_response = input()
    user_response=user_response.lower()
    if(user_response!='bye'):
        if(user_response=='多谢' or user_response=='谢谢' ):
            flag=False
            print("机器人: 不用谢！")
        else:
            if(greeting(user_response)!=None):
                print("机器人: "+greeting(user_response))
                print()
            else:
                print("机器人: ",end="")
                print(response(user_response))
                print()
    else:
        flag=False
        print("小白: 再见! 欢迎再次光临!") 

机器人: 我的名字叫小白. 我可以回答您关于Covid最新政策的问题. 如果您想退出, 请输入:bye !
现在什么政策？
机器人: 对不起,我不理解您的意思,我还是个小白...!

境外输入病例有多少？
机器人: 境外输入现有确诊病例299例（无重症病例），无现有疑似病例。

新增死亡病例有多少？
机器人: 无新增死亡病例。

新增多少？
机器人: 无新增疑似病例。

新增确诊有多少？
机器人: 无新增疑似病例。

上海怎么样？
机器人: 其中境外输入病例31例（广东9例，上海7例，福建5例，内蒙古2例，浙江2例，北京1例，天津1例，辽宁1例，江西1例，河南1例，四川1例），含7例由无症状感染者转为确诊病例（广东4例，天津1例，福建1例，河南1例）；本土病例41例（安徽29例，江苏4例，山东4例，上海2例，福建1例，广东1例），含2例由无症状感染者转为确诊病例（安徽1例，山东1例）。

